In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
import torch
import pandas as pd
import numpy as np
import mlflow
import dagshub
import tensorflow as tf
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from gslTranslater.utils.common import read_yaml, create_directories, save_json
from gslTranslater.constants import *
from urllib.parse import urlparse

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\Dev\\Upwork\\GSL\\GSL-Project'

In [4]:
@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    path_of_model: Path
    test_csv: Path
    mlflow_uri: str
    dagshub_username: str
    dagshub_repo_name: str
    max_seq_length: int
    image_size: list
    batch_size: int
    all_params: dict


In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH, 
        params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation = self.config.evaluation
        data_ingestion = self.config.data_ingestion
        return EvaluationConfig(
            root_dir=Path(data_ingestion.data_dir),
            path_of_model=Path(self.config.training.trained_model_path),
            test_csv=Path(data_ingestion.test_csv),
            mlflow_uri=evaluation.mlflow_uri,
            dagshub_username=evaluation.dagshub_username,
            dagshub_repo_name=evaluation.dagshub_repo_name,
            max_seq_length=self.params.MAX_SEQ_LENGTH,
            image_size=self.params.IMAGE_SIZE,
            batch_size=self.params.BATCH_SIZE,
            all_params=self.params
        )

In [6]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
        self.model = None

    def load_data(self, csv_path):
        df = pd.read_csv(csv_path)

        # Filter for the selected unique words
        df = df[df['Gloss'].isin(df['Gloss'].unique()[:self.config.all_params['NUM_UNIQUE_WORDS']])]

        # Ensure we're selecting only one instance per word
        df = df.groupby('Gloss').head(1).reset_index(drop=True)

        data = []
        labels = []

        for _, row in df.iterrows():
            frames_path = os.path.join(self.config.root_dir, row['Path'])
            frames = sorted([os.path.join(frames_path, img) for img in os.listdir(frames_path) if img.endswith('.jpg')])

            sequence = []
            for frame in frames:
                image = tf.keras.preprocessing.image.load_img(frame, target_size=tuple(self.config.image_size[:-1]))
                image = tf.keras.preprocessing.image.img_to_array(image)
                image = tf.keras.applications.resnet.preprocess_input(image)
                sequence.append(image)

            data.append(sequence)
            labels.append(row['Gloss'])

        # Pad sequences to ensure uniform shape
        data = tf.keras.preprocessing.sequence.pad_sequences(data, maxlen=self.config.max_seq_length, padding='post', dtype='float32')
        labels = np.array(labels)

        return data, labels

    def encode_labels(self, labels):
        label_encoder = LabelEncoder()
        labels = label_encoder.fit_transform(labels)
        labels = torch.tensor(labels)
        labels = nn.functional.one_hot(labels, num_classes=len(np.unique(labels))).float()
        return labels

    def load_model(self, path: Path):
        self.model = tf.keras.models.load_model(path)
        # No need to call self.model.eval() in TensorFlow/Keras

    def evaluate(self):
        # Load and preprocess the data using TensorFlow/Keras
        test_data, test_labels = self.load_data(self.config.test_csv)

        # Encode labels using TensorFlow/Keras utilities
        label_encoder = LabelEncoder()
        test_labels = label_encoder.fit_transform(test_labels)
        test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=len(np.unique(test_labels)))

        # Perform predictions using the Keras model
        predictions = self.model.predict(test_data)

        # Calculate loss and accuracy using TensorFlow/Keras
        loss = tf.keras.losses.CategoricalCrossentropy()(test_labels, predictions)
        accuracy = tf.keras.metrics.CategoricalAccuracy()(test_labels, predictions)

        avg_loss = loss.numpy()
        avg_accuracy = accuracy.numpy()

        print(f'Test Loss: {avg_loss}, Test Accuracy: {avg_accuracy * 100:.2f}%')

        return avg_loss, avg_accuracy


    def save_score(self, avg_loss, avg_accuracy):
        # Convert TensorFlow/PyTorch float32 to standard Python float
        scores = {
            "loss": float(avg_loss),  # Convert to float
            "accuracy": float(avg_accuracy)  # Convert to float
        }
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self, avg_loss, avg_accuracy):
        dagshub.init(repo_owner=self.config.dagshub_username, repo_name=self.config.dagshub_repo_name, mlflow=True)
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"test_loss": avg_loss, "test_accuracy": avg_accuracy})

            # Register the model if not using a file store
            if tracking_url_type_store != "file":
                mlflow.tensorflow.log_model(self.model, "model", registered_model_name="CNN_LSTM_Model")
            else:
                mlflow.tensorflow.log_model(self.model, "model")


In [7]:
try:
    config = ConfigurationManager()
    evaluation_config = config.get_evaluation_config()
    model_evaluation = Evaluation(config=evaluation_config)
    model_evaluation.load_model(evaluation_config.path_of_model)
    avg_loss, avg_accuracy = model_evaluation.evaluate()
    model_evaluation.save_score(avg_loss, avg_accuracy)
    model_evaluation.log_into_mlflow(avg_loss, avg_accuracy)
except Exception as e:
    raise e

[2024-08-30 15:29:23,872: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-30 15:29:23,872: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-30 15:29:23,888: INFO: common: created directory at: artifacts]
1/1 [==============================] - 64s 64s/step
Test Loss: 13.029507637023926, Test Accuracy: 4.00%
[2024-08-30 15:30:45,160: INFO: common: json file saved at: scores.json]


Initialized MLflow to track repo "calvinsendawula/GSL-Project"

[2024-08-30 15:30:46,194: INFO: helpers: Initialized MLflow to track repo "calvinsendawula/GSL-Project"]


Repository calvinsendawula/GSL-Project initialized!

[2024-08-30 15:30:46,194: INFO: helpers: Repository calvinsendawula/GSL-Project initialized!]


2024/08/30 15:30:50 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-08-30 15:31:11,874: WARNING: save: Found untraced functions such as _update_step_xla, lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 56). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\ALIENW~1\AppData\Local\Temp\tmprffzlm52\model\data\model\assets
[2024-08-30 15:31:19,010: INFO: builder_impl: Assets written to: C:\Users\ALIENW~1\AppData\Local\Temp\tmprffzlm52\model\data\model\assets]


Successfully registered model 'CNN_LSTM_Model'.
2024/08/30 15:36:51 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: CNN_LSTM_Model, version 1
Created version '1' of model 'CNN_LSTM_Model'.
